# Using Django and Azure Cosmos DB Graph
## a framework for using Azure Gremlin in the MVC, querying graphs and managing data at web speeds. 

While the Django framework doesn't support graph queries out of the box (It's main design is for SQL databases), It can easily support graph operations using the `gremlinpython` library. Azure's CosmosDB has a gremlin graph database that supports fast read/write operations at very low costs compared to SQL. Together this means 

I've built a tool that facilitates the operations needed to link the two. This means that use CosmosDB as your data source in a Django web app. All of these resources are hosted in Azure. 
* Serverless Web App Service hosting a Python Django App. 
* Azure CosmosDB's Gremlin Graph

![Alt text](../docs/img/infra.png?raw=true "infra")

The Django app needs to be able to solve for these problems:
* Needs to be easy to manage. Simple commands that just run the query and return the data. 
* Connections can't be left open or the server will disconnect, causing the whole app to go down. 
* Opening and closing connetions takes runtime, so I need the funcitonality to open and close connections only as needed. 

The live [application is still under development](http://exodestiny.azurewebsites.net/), but you can see the connection and queries running. 


In [1]:
import sys
sys.path.append('..')

import  ssl, asyncio

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

Most of the `asyncio` and `ssl` stuff is only used for notebooks, it's just here to demo the module. 

In [2]:
from app.models import CosmosdbClient

Note from the import that this is the actual module used in my live web applicaiton. 

The actual `CosmosdbClient` referenced in this [doc is here](../app/models.py).

In [3]:
c = CosmosdbClient()

The module grabs environment variables directly when initiated (`__init__`). You just need to set them when you build them using the webapp service. See my other document on [setting up the app service with your env vars](..\infra\setting-azure-env_vars.md). 

![Alt text](../docs/img/app_service_keys.png?raw=true "infra")

## Run a query:

In [4]:
c.run_query("g.V().haslabel('system').valueMap('hostname','objid','disc_facility','glat','glon').limit(3)")
c.res

[{'hostname': ['PSR B1257+12'],
  'objid': ['26823204'],
  'disc_facility': ['Arecibo Observatory'],
  'glat': ['75.413648'],
  'glon': ['311.30969']},
 {'hostname': ['51 Peg'],
  'objid': ['32247188'],
  'disc_facility': ['Haute-Provence Observatory'],
  'glat': ['-34.72726'],
  'glon': ['90.06264']},
 {'hostname': ['tau Boo'],
  'objid': ['42065158'],
  'disc_facility': ['Roque de los Muchachos Observatory'],
  'glat': ['73.88899'],
  'glon': ['358.9367']}]

The command `run_query` does the following:
* open a connection
* submit the Async query
* get the callback results
* close the connection

This is perfect for quick read/write operations that you'll need in the Ajax functions of your web app, or quick actions in Azure Functions. 

## Run a group of queries

In [5]:
q1 = "g.V().count()"
q2 = "g.V().haslabel('system').count()"
q3 = "g.V().haslabel('pop').count()"

c.add_query(q1)
c.add_query(q2)
c.add_query(q3)

c.run_queries()

In [6]:
c.res

{'g.V().count()': [8812],
 "g.V().haslabel('system').count()": [3245],
 "g.V().haslabel('pop').count()": [988]}

Note that it stores the response in a dict using the query as the key. This makes it easy to fetch your results after running a list of read operations. This is perfect for Django.

In [7]:
print(f"The value of '{q1}' is :{c.res[q1]}",
    f"and the value of '{q2}' is: {c.res[q2]}",
    sep="\n")

The value of 'g.V().count()' is :[8812]
and the value of 'g.V().haslabel('system').count()' is: [3245]


The connection is already closed, so you can focus on your app and your queries and not the connection status. 

## Cleaning the data
Just basic python at this point. I've added some tools to help turn the data into a dict, like object suited for Ajax requests. But you can ignore this entierely. 

In [8]:
c.run_query("g.V().haslabel('system').valueMap('hostname','objid','disc_facility','glat','glon').limit(3)")
c.res

[{'hostname': ['PSR B1257+12'],
  'objid': ['26823204'],
  'disc_facility': ['Arecibo Observatory'],
  'glat': ['75.413648'],
  'glon': ['311.30969']},
 {'hostname': ['51 Peg'],
  'objid': ['32247188'],
  'disc_facility': ['Haute-Provence Observatory'],
  'glat': ['-34.72726'],
  'glon': ['90.06264']},
 {'hostname': ['tau Boo'],
  'objid': ['42065158'],
  'disc_facility': ['Roque de los Muchachos Observatory'],
  'glat': ['73.88899'],
  'glon': ['358.9367']}]

In [9]:
c.clean_nodes(c.res)

[{'hostname': 'PSR B1257+12',
  'objid': '26823204',
  'disc_facility': 'Arecibo Observatory',
  'glat': '75.413648',
  'glon': '311.30969',
  'id': '26823204'},
 {'hostname': '51 Peg',
  'objid': '32247188',
  'disc_facility': 'Haute-Provence Observatory',
  'glat': '-34.72726',
  'glon': '90.06264',
  'id': '32247188'},
 {'hostname': 'tau Boo',
  'objid': '42065158',
  'disc_facility': 'Roque de los Muchachos Observatory',
  'glat': '73.88899',
  'glon': '358.9367',
  'id': '42065158'}]

# Getting the query metadata

I haven't implemented advanced metadata logging, but probably should. This is how it _would_ be done.

In [13]:
from gremlin_python.driver import client,serializer
import os 
c = client.Client(
                os.getenv("endpoint","env vars not set"),
                "g",
                username=os.getenv("dbusername","env vars not set"),
                password=os.getenv("dbkey","env vars not set")+"==",
                message_serializer=serializer.GraphSONSerializersV2d0(),
            )

In [35]:
callback = c.submitAsync("g.V().count()")

In [36]:
status = callback.result().status_attributes

In [37]:
callback.result().all().result()

[8812]

In [39]:
str(status)

"{'x-ms-status-code': 200, 'x-ms-activity-id': 'd9899238-915c-4b68-a8e0-a0e2364b97ee', 'x-ms-request-charge': 3.19, 'x-ms-total-request-charge': 3.19, 'x-ms-server-time-ms': 19.4287, 'x-ms-total-server-time-ms': 19.4287}"

## Limitations

* This is for quick read, write operations. Thing single function web apps like Flask, Django, Azure Functions
* This is not for high volume or bulk uploads. You should consider other approaches. 
* The cleaning tools are fit to purpose for my application, you may need to modify them for your app. 
